### Akshit Arora
Adopted from : https://github.com/eisenjulian/nlp_estimator_tutorial/blob/master/nlp_estimators.ipynb

In [1]:
import os
import string
import tempfile
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk, re, pprint
from nltk import word_tokenize
from nltk.corpus import stopwords
import string
from collections import Counter
from nltk.stem import PorterStemmer, WordNetLemmatizer

from tensorflow.python.keras.preprocessing import sequence
from tensorboard import summary as summary_lib

tf.logging.set_verbosity(tf.logging.INFO)
print(tf.__version__)

1.6.0


## Loading the data

In [2]:
DATA_PATH = "../../data/wikipedia/train.csv"
label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
comment_col = 'comment_text'
stop = stopwords.words('english') + list(string.punctuation) + list(string.printable)

In [3]:
dF = pd.read_csv(DATA_PATH)

In [100]:
def tokenize(dataFrame, porterEnable = False):
    '''
    Tokenizes the sentence, removes stopwords and applied stemming to comments
    '''
    #dataFrame['tokenized_comment'] = dataFrame['comment_text'].apply(word_tokenize)
    dataFrame['tokenized_comment'] = dataFrame['comment_text'].apply(lambda x : filter(None,x.lower().strip().translate([None, string.punctuation]).split(" ")))
    dataFrame['tokenized_comment'] = dataFrame['tokenized_comment'].apply(lambda x: [item for item in x if item not in stop and item.isalpha()])
    #apply stemming
    if(porterEnable):
        porter = nltk.PorterStemmer()
        dataFrame['tokenized_comment'] = dataFrame['tokenized_comment'].apply(lambda x : [porter.stem(y) for y in x])
    return dataFrame

def prepareWordDict(df, limit):
    '''
    Prepares a dict of words from tokenized_comment column with their frequencies as keys. 
    Then truncates it to limit on vocabulary
    '''
    word_dict = Counter()
    for i in df['tokenized_comment']: #go per comment
        for j in i: #go per word
            word_dict[str(j)] += 1
    trunc_dict = dict(word_dict.most_common(limit-3))
    index = 3
    for key in trunc_dict: #assign unique keys to all words
        trunc_dict[key] = index
        index += 1
    return trunc_dict
    
def genSeq(df, wordDict, startId, unkId):
    '''
    generates variable length sequences based on wordDict
    '''
    sequence = []
    y = {}
    y2 = []
    for i in label_cols:
        y[i] = []
    
    for index, row in df.iterrows(): #go per comment
        temp = []
        temp.append(startId)
        for j in row['tokenized_comment']: #go per word in the list
            if(j not in wordDict):
                temp.append(unkId)
            else:
                temp.append(wordDict[j])
        temp_y2 = []
        for k in label_cols:
            y[k].append(row[k])
            temp_y2.append(row[k])
        y2.append(temp_y2)
        sequence.append(temp)
    return sequence, y, y2
    
def prepareSequences(dataFrame, vocabSize, split = 0.8, startId = 1, unkId = 2):
    '''
    prepares variable length sequences from the sentences to feed to pad_sequences function of keras
    args:
    dataFrame = imported from csv file
    vocabSize = Limit to number of words in vocabulary
    split = the fraction to take as training set
    startId = id that indicates start of the sentence
    unkId = id that indicates words out of vocabulary
    '''
    dataFrame = dataFrame.sample(frac=1).reset_index(drop=True) #shuffle df rows
    dataFrame = tokenize(dataFrame) #create the tokenized comment column
    wordDict = prepareWordDict(dataFrame, limit=vocabSize)
    
    #split the data
    msk = np.random.rand(len(dataFrame)) < split
    train = dataFrame[msk]
    test = dataFrame[~msk]
    
    #generate train sequences
    train_sequence, y_train, y2_train = genSeq(train, wordDict, startId, unkId)  
    
    #prepare test sequences
    test_sequence, y_test, y2_test = genSeq(test, wordDict, startId, unkId)
    
    return train_sequence, y_train, y2_train, test_sequence, y_test, y2_test, wordDict

In [101]:
vocab_size = 5000
sentence_size = 200
embedding_size = 50
pad_id = 0

print("Loading data...")

x_train_variable, y_train, y2_train, x_test_variable, y_test, y2_test, word_dict = prepareSequences(dataFrame = dF, vocabSize = vocab_size)

for i in label_cols:
    print(len(y_train[i]), "train sequences")
    print(len(y_test[i]), "test sequences")
    break

print("Pad sequences (samples x time)")
x_train = sequence.pad_sequences(x_train_variable, 
                                 maxlen=sentence_size,
                                 truncating='post',
                                 padding='post',
                                 value=pad_id)
x_test = sequence.pad_sequences(x_test_variable, 
                                maxlen=sentence_size,
                                truncating='post',
                                padding='post', 
                                value=pad_id)
print("x_train shape:", x_train.shape)
print("x_test shape:", x_test.shape)

Loading data...
127697 train sequences
31874 test sequences
Pad sequences (samples x time)
x_train shape: (127697, 200)
x_test shape: (31874, 200)


Let's inspect one of the comments.

In [109]:
pad_id = 0
start_id = 1
oov_id = 2 #same as unkId

word_inverted_index = {v: k for k, v in word_dict.items()}

# The first indexes in the map are reserved to represent things other than tokens
word_inverted_index[pad_id] = '<PAD>'
word_inverted_index[start_id] = '<START>'
word_inverted_index[oov_id] = '<OOV>'

for i in range(0, 10):
  print(i, word_inverted_index[i])
  
def index_to_text(indexes):
    return ' '.join([word_inverted_index[i] for i in indexes])

print(index_to_text(x_train_variable[0]))

0 <PAD>
1 <START>
2 <OOV>
3 article
4 would
5 page
6 like
7 one
8 please
9 talk
<START> edits <OOV> edits wikipedia alone <OOV> believe <OOV> edits agree information provided article edit contributions unless actually knowledgeable subject


## Building Estimators

### From arrays to tensors

In [117]:
x_len_train = np.array([min(len(x), sentence_size) for x in x_train_variable])
x_len_test = np.array([min(len(x), sentence_size) for x in x_test_variable])
model_dir = {}
for i in label_cols:
    model_dir[i] = tempfile.mkdtemp()

def parser(x, length, y):
    features = {"x": x, "len": length}
    return features, y

def train_input_fn(modelOutput = 'identity_hate'):
    dataset = tf.data.Dataset.from_tensor_slices((x_train, x_len_train, y_train[modelOutput]))
    dataset = dataset.shuffle(buffer_size=len(x_train_variable))
    dataset = dataset.batch(100)
    dataset = dataset.map(parser)
    dataset = dataset.repeat()
    iterator = dataset.make_one_shot_iterator()
    return iterator.get_next()

def eval_input_fn(modelOutput = 'identity_hate'):
    dataset = tf.data.Dataset.from_tensor_slices((x_test, x_len_test, y_test[modelOutput]))
    dataset = dataset.batch(100)
    dataset = dataset.map(parser)
    iterator = dataset.make_one_shot_iterator()
    return iterator.get_next()

### Model 1: Linear Classifier

In [118]:
column = tf.feature_column.categorical_column_with_identity('x', num_buckets = vocab_size)
classifier = {}
for i in label_cols:
    classifier[i] = tf.estimator.LinearClassifier(feature_columns=[column], model_dir=os.path.join(model_dir[i], 'bow_sparse'))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/q4/23xs17696m12p8g5htyy6nyh0000gn/T/tmpuq3dxdwx/bow_sparse', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x139459f60>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/q4/23xs17696m12p8g5htyy6nyh0000gn/T/tmp4jwcf602/bow_sparse', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoi

In [119]:
all_classifiers = {}
def train_and_evaluate(classifier, modelOutput):
    # Save a reference to the classifier to run predictions later
    all_classifiers[classifier.model_dir] = classifier
    classifier.train(input_fn=lambda:train_input_fn(modelOutput), steps=25000)
    eval_results = classifier.evaluate(input_fn=lambda:eval_input_fn(modelOutput))
    predictions = np.array([p['logistic'][0] for p in classifier.predict(input_fn=lambda:eval_input_fn(modelOutput))])
        
    # Reset the graph to be able to reuse name scopes
    tf.reset_default_graph() 
    # Add a PR summary in addition to the summaries that the classifier writes
    pr = summary_lib.pr_curve('precision_recall', predictions=predictions, labels=np.array(y_test[modelOutput]).astype(bool), num_thresholds=21)
    with tf.Session() as sess:
        writer = tf.summary.FileWriter(os.path.join(classifier.model_dir, 'eval'), sess.graph)
        print("Saving to: "+str(os.path.join(classifier.model_dir, 'eval')))
        writer.add_summary(sess.run(pr), global_step=0)
        writer.close()

In [120]:
for i in label_cols:
    print("For model: "+str(i))
    train_and_evaluate(classifier[i],i)

For model: toxic
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/q4/23xs17696m12p8g5htyy6nyh0000gn/T/tmpuq3dxdwx/bow_sparse/model.ckpt.
INFO:tensorflow:loss = 69.31472, step = 1
INFO:tensorflow:global_step/sec: 114.634
INFO:tensorflow:loss = 16.196249, step = 101 (0.873 sec)
INFO:tensorflow:global_step/sec: 363.546
INFO:tensorflow:loss = 12.912735, step = 201 (0.275 sec)
INFO:tensorflow:global_step/sec: 393.613
INFO:tensorflow:loss = 17.002491, step = 301 (0.254 sec)
INFO:tensorflow:global_step/sec: 379.106
INFO:tensorflow:loss = 44.357315, step = 401 (0.264 sec)
INFO:tensorflow:global_step/sec: 354.74
INFO:tensorflow:loss = 15.109918, step = 501 (0.282 sec)
INFO:tensorflow:global_step/sec: 343.768
INFO:tensorflow:loss = 21.841152, step = 601

INFO:tensorflow:global_step/sec: 338.139
INFO:tensorflow:loss = 10.610393, step = 8001 (0.296 sec)
INFO:tensorflow:global_step/sec: 259.228
INFO:tensorflow:loss = 14.756354, step = 8101 (0.386 sec)
INFO:tensorflow:global_step/sec: 319.666
INFO:tensorflow:loss = 12.704987, step = 8201 (0.313 sec)
INFO:tensorflow:global_step/sec: 298.147
INFO:tensorflow:loss = 16.188766, step = 8301 (0.335 sec)
INFO:tensorflow:global_step/sec: 327.693
INFO:tensorflow:loss = 18.286407, step = 8401 (0.305 sec)
INFO:tensorflow:global_step/sec: 317.384
INFO:tensorflow:loss = 21.623922, step = 8501 (0.316 sec)
INFO:tensorflow:global_step/sec: 348.711
INFO:tensorflow:loss = 15.317748, step = 8601 (0.286 sec)
INFO:tensorflow:global_step/sec: 348.091
INFO:tensorflow:loss = 12.032462, step = 8701 (0.288 sec)
INFO:tensorflow:global_step/sec: 337.248
INFO:tensorflow:loss = 17.327103, step = 8801 (0.296 sec)
INFO:tensorflow:global_step/sec: 334.93
INFO:tensorflow:loss = 13.0046, step = 8901 (0.298 sec)
INFO:tensorfl

INFO:tensorflow:global_step/sec: 355.03
INFO:tensorflow:loss = 13.340221, step = 16301 (0.282 sec)
INFO:tensorflow:global_step/sec: 340.825
INFO:tensorflow:loss = 10.576806, step = 16401 (0.293 sec)
INFO:tensorflow:global_step/sec: 336.657
INFO:tensorflow:loss = 19.058964, step = 16501 (0.297 sec)
INFO:tensorflow:global_step/sec: 334.934
INFO:tensorflow:loss = 13.969003, step = 16601 (0.299 sec)
INFO:tensorflow:global_step/sec: 220.805
INFO:tensorflow:loss = 10.202274, step = 16701 (0.453 sec)
INFO:tensorflow:global_step/sec: 380.709
INFO:tensorflow:loss = 15.34822, step = 16801 (0.263 sec)
INFO:tensorflow:global_step/sec: 378.115
INFO:tensorflow:loss = 13.325353, step = 16901 (0.264 sec)
INFO:tensorflow:global_step/sec: 361.96
INFO:tensorflow:loss = 12.354478, step = 17001 (0.276 sec)
INFO:tensorflow:global_step/sec: 362.495
INFO:tensorflow:loss = 12.921873, step = 17101 (0.276 sec)
INFO:tensorflow:global_step/sec: 381.8
INFO:tensorflow:loss = 11.371396, step = 17201 (0.262 sec)
INFO:

INFO:tensorflow:loss = 10.623878, step = 24501 (0.278 sec)
INFO:tensorflow:global_step/sec: 375.934
INFO:tensorflow:loss = 12.375661, step = 24601 (0.266 sec)
INFO:tensorflow:global_step/sec: 364.788
INFO:tensorflow:loss = 5.3237104, step = 24701 (0.274 sec)
INFO:tensorflow:global_step/sec: 372.541
INFO:tensorflow:loss = 21.86323, step = 24801 (0.268 sec)
INFO:tensorflow:global_step/sec: 373.028
INFO:tensorflow:loss = 14.835948, step = 24901 (0.268 sec)
INFO:tensorflow:Saving checkpoints for 25000 into /var/folders/q4/23xs17696m12p8g5htyy6nyh0000gn/T/tmpuq3dxdwx/bow_sparse/model.ckpt.
INFO:tensorflow:Loss for final step: 12.54078.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-04-29-04:23:26
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/q4/23xs17696m12p8g5htyy6nyh0000gn/T/tmpuq3dxdwx/bow_sparse/model.ckpt-25000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done r

INFO:tensorflow:global_step/sec: 346.741
INFO:tensorflow:loss = 2.5517995, step = 6201 (0.288 sec)
INFO:tensorflow:global_step/sec: 305.539
INFO:tensorflow:loss = 2.0653827, step = 6301 (0.328 sec)
INFO:tensorflow:global_step/sec: 198.874
INFO:tensorflow:loss = 0.3877608, step = 6401 (0.503 sec)
INFO:tensorflow:global_step/sec: 364.957
INFO:tensorflow:loss = 4.128075, step = 6501 (0.274 sec)
INFO:tensorflow:global_step/sec: 347.319
INFO:tensorflow:loss = 7.8881946, step = 6601 (0.288 sec)
INFO:tensorflow:global_step/sec: 372.757
INFO:tensorflow:loss = 0.24281612, step = 6701 (0.268 sec)
INFO:tensorflow:global_step/sec: 355.782
INFO:tensorflow:loss = 3.6117322, step = 6801 (0.281 sec)
INFO:tensorflow:global_step/sec: 363.07
INFO:tensorflow:loss = 0.88285404, step = 6901 (0.275 sec)
INFO:tensorflow:global_step/sec: 350.11
INFO:tensorflow:loss = 1.6947854, step = 7001 (0.286 sec)
INFO:tensorflow:global_step/sec: 366.274
INFO:tensorflow:loss = 2.596031, step = 7101 (0.273 sec)
INFO:tensorf

...

INFO:tensorflow:global_step/sec: 324.224
INFO:tensorflow:loss = 11.722387, step = 17201 (0.308 sec)
INFO:tensorflow:global_step/sec: 176.893
INFO:tensorflow:loss = 6.2242, step = 17301 (0.566 sec)
INFO:tensorflow:global_step/sec: 161.137
INFO:tensorflow:loss = 8.145507, step = 17401 (0.622 sec)
INFO:tensorflow:global_step/sec: 147.651
INFO:tensorflow:loss = 7.2915626, step = 17501 (0.676 sec)
INFO:tensorflow:global_step/sec: 207.904
INFO:tensorflow:loss = 10.773315, step = 17601 (0.482 sec)
INFO:tensorflow:global_step/sec: 206.441
INFO:tensorflow:loss = 5.1796436, step = 17701 (0.482 sec)
INFO:tensorflow:global_step/sec: 205.066
INFO:tensorflow:loss = 8.0620985, step = 17801 (0.488 sec)
INFO:tensorflow:global_step/sec: 99.4945
INFO:tensorflow:loss = 2.7017562, step = 17901 (1.006 sec)
INFO:tensorflow:global_step/sec: 179.293
INFO:tensorflow:loss = 6.4298573, step = 18001 (0.558 sec)
INFO:tensorflow:global_step/sec: 232.08
INFO:tensorflow:loss = 7.273476, step = 18101 (0.436 sec)
INFO:t

INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-04-29-04:33:02
INFO:tensorflow:Saving dict for global step 25000: accuracy = 0.96103406, accuracy_baseline = 0.9516848, auc = 0.90045893, auc_precision_recall = 0.52386475, average_loss = 0.21124326, global_step = 25000, label/mean = 0.048315242, loss = 21.107107, prediction/mean = 0.046926267
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/q4/23xs17696m12p8g5htyy6nyh0000gn/T/tmpki54rdcp/bow_sparse/model.ckpt-25000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Saving to: /var/folders/q4/23xs17696m12p8g5htyy6nyh0000gn/T/tmpki54rdcp/bow_sparse/eval
For model: identity_hate
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph wa

INFO:tensorflow:loss = 12.011526, step = 7101 (0.314 sec)
INFO:tensorflow:global_step/sec: 332.556
INFO:tensorflow:loss = 6.172925, step = 7201 (0.300 sec)
INFO:tensorflow:global_step/sec: 317.309
INFO:tensorflow:loss = 0.7611202, step = 7301 (0.315 sec)
INFO:tensorflow:global_step/sec: 329.988
INFO:tensorflow:loss = 2.7770314, step = 7401 (0.303 sec)
INFO:tensorflow:global_step/sec: 310.348
INFO:tensorflow:loss = 2.6598005, step = 7501 (0.322 sec)
INFO:tensorflow:global_step/sec: 316.009
INFO:tensorflow:loss = 9.186984, step = 7601 (0.317 sec)
INFO:tensorflow:global_step/sec: 197.479
INFO:tensorflow:loss = 0.2678448, step = 7701 (0.506 sec)
INFO:tensorflow:global_step/sec: 344.234
INFO:tensorflow:loss = 4.712261, step = 7801 (0.291 sec)
INFO:tensorflow:global_step/sec: 339.16
INFO:tensorflow:loss = 3.4166443, step = 7901 (0.295 sec)
INFO:tensorflow:global_step/sec: 352.413
INFO:tensorflow:loss = 0.44265231, step = 8001 (0.284 sec)
INFO:tensorflow:global_step/sec: 323.86
INFO:tensorflo

INFO:tensorflow:global_step/sec: 198.786
INFO:tensorflow:loss = 3.8872237, step = 15401 (0.503 sec)
INFO:tensorflow:global_step/sec: 344.156
INFO:tensorflow:loss = 0.94830245, step = 15501 (0.291 sec)
INFO:tensorflow:global_step/sec: 341.407
INFO:tensorflow:loss = 0.56804585, step = 15601 (0.292 sec)
INFO:tensorflow:global_step/sec: 314.425
INFO:tensorflow:loss = 0.8429262, step = 15701 (0.318 sec)
INFO:tensorflow:global_step/sec: 339.81
INFO:tensorflow:loss = 0.87572443, step = 15801 (0.295 sec)
INFO:tensorflow:global_step/sec: 321.807
INFO:tensorflow:loss = 3.7122188, step = 15901 (0.310 sec)
INFO:tensorflow:global_step/sec: 311.251
INFO:tensorflow:loss = 5.684397, step = 16001 (0.321 sec)
INFO:tensorflow:global_step/sec: 312.983
INFO:tensorflow:loss = 3.635824, step = 16101 (0.319 sec)
INFO:tensorflow:global_step/sec: 333.346
INFO:tensorflow:loss = 2.6869314, step = 16201 (0.300 sec)
INFO:tensorflow:global_step/sec: 318.977
INFO:tensorflow:loss = 1.0463482, step = 16301 (0.314 sec)


INFO:tensorflow:loss = 0.2635877, step = 23601 (0.300 sec)
INFO:tensorflow:global_step/sec: 316.431
INFO:tensorflow:loss = 0.15857007, step = 23701 (0.316 sec)
INFO:tensorflow:global_step/sec: 339.999
INFO:tensorflow:loss = 2.6821053, step = 23801 (0.294 sec)
INFO:tensorflow:global_step/sec: 328.501
INFO:tensorflow:loss = 1.778731, step = 23901 (0.304 sec)
INFO:tensorflow:global_step/sec: 326.024
INFO:tensorflow:loss = 2.448586, step = 24001 (0.307 sec)
INFO:tensorflow:global_step/sec: 324.68
INFO:tensorflow:loss = 0.5139609, step = 24101 (0.308 sec)
INFO:tensorflow:global_step/sec: 298.551
INFO:tensorflow:loss = 1.3972362, step = 24201 (0.335 sec)
INFO:tensorflow:global_step/sec: 203.317
INFO:tensorflow:loss = 4.6697235, step = 24301 (0.492 sec)
INFO:tensorflow:global_step/sec: 361.161
INFO:tensorflow:loss = 0.24889746, step = 24401 (0.277 sec)
INFO:tensorflow:global_step/sec: 350.087
INFO:tensorflow:loss = 0.5486735, step = 24501 (0.286 sec)
INFO:tensorflow:global_step/sec: 359.451
I

In [123]:
for idx, i in enumerate(label_cols):
    print(str(i))
    weights = classifier[i].get_variable_value('linear/linear_model/x/weights').flatten()
    sorted_indexes = np.argsort(weights)
    extremes = np.concatenate((sorted_indexes[-8:], sorted_indexes[:8]))
    extreme_weights = sorted([(weights[i], word_inverted_index[i]) for i in extremes])
    y_pos = np.arange(len(extreme_weights))
    plt.bar(y_pos, [pair[0] for pair in extreme_weights], align='center', alpha=0.5)
    plt.xticks(y_pos, [pair[1] for pair in extreme_weights], rotation=45, ha='right')
    plt.ylabel('Weight')
    plt.title('Most significant tokens') 
    plt.show()

toxic


TypeError: 'DNNClassifier' object is not subscriptable

### Model 2: DNN Classifier with Embeddings

In [124]:
word_embedding_column = tf.feature_column.embedding_column(column, dimension=embedding_size)
for i in label_cols:
    print("For model: "+str(i))
    classifier = tf.estimator.DNNClassifier(hidden_units=[100], feature_columns=[word_embedding_column], model_dir=os.path.join(model_dir[i], 'bow_embeddings'))
    train_and_evaluate(classifier,i)

In [ ]:
# for i in label_cols:
#     with open(os.path.join(model_dir[i], str(i)+'_metadata.tsv'), 'w', encoding="utf-8") as f:
#         f.write('label\n')
#         for index in range(0, vocab_size):
#             f.write(word_inverted_index[index] + '\t')
#         f.write('\n')
#     print("mv " +os.path.join(model_dir[i], str(i)+'_metadata.tsv')+" ~/Desktop/")
    
# will visualize using http://projector.tensorflow.org/ tutorial: https://www.tensorflow.org/programmers_guide/embedding ; We can use TensorBoard to visualize our  5050 -dimensional word vectors projected into  ℝ3R3  using t-SNE. We expect similar word to be close to each other. This can be a useful way to inspect our model weights and find unexpected behaviours. There's plenty of more information to go deeper here. The following snippet will generate a vocabulary file metadata.tsv that lists all the tokens in order. In the PROJECTOR tab in TensorBoard you can load it to visualize your vectors and there's also the standalone projector visualizer that can be used to check out different embeddings.

### Model 3: Convolutional Neural Network

In [ ]:
head = tf.contrib.estimator.binary_classification_head()

def cnn_model_fn(features, labels, mode, params):    
    input_layer = tf.contrib.layers.embed_sequence(
        features['x'], vocab_size, embedding_size, initializer=params['embedding_initializer'])
    
    training = mode == tf.estimator.ModeKeys.TRAIN
    dropout_emb = tf.layers.dropout(inputs=input_layer, 
                                    rate=0.2, 
                                    training=training)

    conv = tf.layers.conv1d(
        inputs=dropout_emb,
        filters=32,
        kernel_size=3,
        padding="same",
        activation=tf.nn.relu)
    
    # Global Max Pooling
    pool = tf.reduce_max(input_tensor=conv, axis=1)
    
    hidden = tf.layers.dense(inputs=pool, units=250, activation=tf.nn.relu)
    
    dropout_hidden = tf.layers.dropout(inputs=hidden, 
                                       rate=0.2, 
                                       training=training)
    
    logits = tf.layers.dense(inputs=dropout_hidden, units=1)
    
    # This will be None when predicting
    if labels is not None:
        labels = tf.reshape(labels, [-1, 1])
        

    optimizer = tf.train.AdamOptimizer()
    
    def _train_op_fn(loss):
        return optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())

    return head.create_estimator_spec(
        features=features,
        labels=labels,
        mode=mode,
        logits=logits, 
        train_op_fn=_train_op_fn)

params = {}
cnn_classifier = {}
for i in label_cols:
    print("For model: "+str(i))
    params[i] = {'embedding_initializer': tf.random_uniform_initializer(-1.0, 1.0)}
    cnn_classifier[i] = tf.estimator.Estimator(model_fn=cnn_model_fn,model_dir=os.path.join(model_dir[i], 'cnn'),params=params[i])
    train_and_evaluate(cnn_classifier[i],i)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/q4/23xs17696m12p8g5htyy6nyh0000gn/T/tmpuq3dxdwx/cnn', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1419a2208>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
IN

INFO:tensorflow:global_step/sec: 18.7226
INFO:tensorflow:loss = 11.97373, step = 7001 (5.341 sec)
INFO:tensorflow:global_step/sec: 22.2096
INFO:tensorflow:loss = 14.289096, step = 7101 (4.503 sec)
INFO:tensorflow:global_step/sec: 19.9298
INFO:tensorflow:loss = 14.66041, step = 7201 (5.017 sec)
INFO:tensorflow:global_step/sec: 20.6828
INFO:tensorflow:loss = 10.751409, step = 7301 (4.834 sec)
INFO:tensorflow:global_step/sec: 22.679
INFO:tensorflow:loss = 13.869459, step = 7401 (4.409 sec)
INFO:tensorflow:global_step/sec: 21.1535
INFO:tensorflow:loss = 11.429649, step = 7501 (4.727 sec)
INFO:tensorflow:global_step/sec: 19.4285
INFO:tensorflow:loss = 10.596785, step = 7601 (5.147 sec)
INFO:tensorflow:global_step/sec: 22.6303
INFO:tensorflow:loss = 14.202995, step = 7701 (4.419 sec)
INFO:tensorflow:global_step/sec: 20.2807
INFO:tensorflow:loss = 14.6422825, step = 7801 (4.931 sec)
INFO:tensorflow:global_step/sec: 21.8265
INFO:tensorflow:loss = 16.324165, step = 7901 (4.582 sec)
INFO:tensorf

INFO:tensorflow:loss = 13.275856, step = 15101 (4.172 sec)
INFO:tensorflow:global_step/sec: 24.7288
INFO:tensorflow:loss = 8.389461, step = 15201 (4.043 sec)
INFO:tensorflow:global_step/sec: 21.1867
INFO:tensorflow:loss = 13.538239, step = 15301 (4.720 sec)
INFO:tensorflow:global_step/sec: 22.1339
INFO:tensorflow:loss = 7.716434, step = 15401 (4.518 sec)
INFO:tensorflow:global_step/sec: 25.2025
INFO:tensorflow:loss = 8.721451, step = 15501 (3.968 sec)
INFO:tensorflow:global_step/sec: 24.8017
INFO:tensorflow:loss = 8.347841, step = 15601 (4.032 sec)
INFO:tensorflow:global_step/sec: 17.8461
INFO:tensorflow:loss = 8.275721, step = 15701 (5.603 sec)
INFO:tensorflow:global_step/sec: 18.7865
INFO:tensorflow:loss = 8.754384, step = 15801 (5.326 sec)
INFO:tensorflow:global_step/sec: 16.9842
INFO:tensorflow:loss = 7.3179374, step = 15901 (5.885 sec)
INFO:tensorflow:global_step/sec: 19.0126
INFO:tensorflow:loss = 5.302883, step = 16001 (5.259 sec)
INFO:tensorflow:global_step/sec: 25.266
INFO:ten

INFO:tensorflow:global_step/sec: 20.7806
INFO:tensorflow:loss = 10.3580885, step = 23401 (4.812 sec)
INFO:tensorflow:global_step/sec: 25.0145
INFO:tensorflow:loss = 13.242524, step = 23501 (3.998 sec)
INFO:tensorflow:global_step/sec: 25.2463
INFO:tensorflow:loss = 5.7069197, step = 23601 (3.961 sec)
INFO:tensorflow:global_step/sec: 25.2219
INFO:tensorflow:loss = 11.485009, step = 23701 (3.965 sec)
INFO:tensorflow:global_step/sec: 25.1268
INFO:tensorflow:loss = 6.8866405, step = 23801 (3.980 sec)
INFO:tensorflow:global_step/sec: 24.067
INFO:tensorflow:loss = 6.629845, step = 23901 (4.155 sec)
INFO:tensorflow:global_step/sec: 24.8202
INFO:tensorflow:loss = 10.302494, step = 24001 (4.029 sec)
INFO:tensorflow:global_step/sec: 25.0673
INFO:tensorflow:loss = 7.2702007, step = 24101 (3.990 sec)
INFO:tensorflow:global_step/sec: 24.5679
INFO:tensorflow:loss = 5.206851, step = 24201 (4.070 sec)
INFO:tensorflow:global_step/sec: 20.0063
INFO:tensorflow:loss = 4.5868535, step = 24301 (4.998 sec)
IN

INFO:tensorflow:global_step/sec: 22.5217
INFO:tensorflow:loss = 0.333725, step = 4401 (4.440 sec)
INFO:tensorflow:global_step/sec: 19.2265
INFO:tensorflow:loss = 2.7476773, step = 4501 (5.201 sec)
INFO:tensorflow:global_step/sec: 20.8126
INFO:tensorflow:loss = 1.0006092, step = 4601 (4.805 sec)
INFO:tensorflow:global_step/sec: 18.9515
INFO:tensorflow:loss = 1.8406234, step = 4701 (5.277 sec)
INFO:tensorflow:global_step/sec: 23.2183
INFO:tensorflow:loss = 3.9213564, step = 4801 (4.307 sec)
INFO:tensorflow:global_step/sec: 19.2063
INFO:tensorflow:loss = 0.15950327, step = 4901 (5.207 sec)
INFO:tensorflow:global_step/sec: 19.4845
INFO:tensorflow:loss = 0.5294669, step = 5001 (5.134 sec)
INFO:tensorflow:global_step/sec: 18.3229
INFO:tensorflow:loss = 0.4715748, step = 5101 (5.456 sec)
INFO:tensorflow:global_step/sec: 19.625
INFO:tensorflow:loss = 0.33131403, step = 5201 (5.095 sec)
INFO:tensorflow:global_step/sec: 24.8082
INFO:tensorflow:loss = 4.0783916, step = 5301 (4.031 sec)
INFO:tenso

INFO:tensorflow:global_step/sec: 18.5605
INFO:tensorflow:loss = 2.2852383, step = 12701 (5.388 sec)
INFO:tensorflow:global_step/sec: 19.3559
INFO:tensorflow:loss = 1.3889163, step = 12801 (5.167 sec)
INFO:tensorflow:global_step/sec: 20.2768
INFO:tensorflow:loss = 0.23261039, step = 12901 (4.931 sec)
INFO:tensorflow:global_step/sec: 19.8392
INFO:tensorflow:loss = 0.4857406, step = 13001 (5.041 sec)
INFO:tensorflow:global_step/sec: 20.2001
INFO:tensorflow:loss = 0.8178548, step = 13101 (4.950 sec)
INFO:tensorflow:global_step/sec: 20.4234
INFO:tensorflow:loss = 0.11041053, step = 13201 (4.896 sec)
INFO:tensorflow:global_step/sec: 20.4008
INFO:tensorflow:loss = 2.3271677, step = 13301 (4.902 sec)
INFO:tensorflow:global_step/sec: 20.3214
INFO:tensorflow:loss = 0.1530806, step = 13401 (4.921 sec)
INFO:tensorflow:global_step/sec: 20.3555
INFO:tensorflow:loss = 1.3193482, step = 13501 (4.913 sec)
INFO:tensorflow:global_step/sec: 20.0939
INFO:tensorflow:loss = 0.022408962, step = 13601 (4.976 s

INFO:tensorflow:global_step/sec: 23.4136
INFO:tensorflow:loss = 0.98761016, step = 20801 (4.271 sec)
INFO:tensorflow:global_step/sec: 22.5344
INFO:tensorflow:loss = 0.007668529, step = 20901 (4.438 sec)
INFO:tensorflow:global_step/sec: 24.1527
INFO:tensorflow:loss = 2.2343447, step = 21001 (4.140 sec)
INFO:tensorflow:global_step/sec: 23.4803
INFO:tensorflow:loss = 4.973806, step = 21101 (4.259 sec)
INFO:tensorflow:global_step/sec: 22.3007
INFO:tensorflow:loss = 1.649796, step = 21201 (4.484 sec)
INFO:tensorflow:global_step/sec: 22.4675
INFO:tensorflow:loss = 0.1350521, step = 21301 (4.451 sec)
INFO:tensorflow:global_step/sec: 21.1673
INFO:tensorflow:loss = 4.810625, step = 21401 (4.724 sec)
INFO:tensorflow:global_step/sec: 23.4
INFO:tensorflow:loss = 0.1960198, step = 21501 (4.273 sec)
INFO:tensorflow:global_step/sec: 22.7683
INFO:tensorflow:loss = 1.1371224, step = 21601 (4.392 sec)
INFO:tensorflow:global_step/sec: 23.2006
INFO:tensorflow:loss = 0.032059222, step = 21701 (4.310 sec)
I